# Notebook 2 - SQL avec vraies bases de données
## Analyse e-commerce avec PostgreSQL en ligne




### 🎯 Objectifs pédagogiques
- Connecter Python à une vraie base de données PostgreSQL
- Écrire des requêtes SQL complexes sur des données réelles
- Implémenter des analyses RFM avec SQL
- Intégrer SQL et pandas pour des analyses avancées
- Gérer les connexions et la sécurité

### 🛍️ Contexte du projet
Vous analysez les données d'un vrai dataset e-commerce (Brazilian E-Commerce Public Dataset) hébergé sur une base PostgreSQL.

Objectif : créer une segmentation clientèle pour optimiser les campagnes marketing.


## Partie 1 : Connexion à la base de données réelle

### 🔧 Installation et configuration


# Installation des dépendances


```
pip install psycopg2-binary sqlalchemy pandas python-dotenv
```




In [1]:
import psycopg2
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

### 🌐 Base de données PostgreSQL gratuite (ElephantSQL)

**Option 1 : ElephantSQL (20MB gratuit)**
1. Créez un compte sur [elephantsql.com](https://www.elephantsql.com/)
2. Créez une instance "Tiny Turtle" (gratuite)
3. Récupérez vos credentials

**Option 2 : Supabase (500MB gratuit)**
1. Créez un compte sur [supabase.com](https://supabase.com/)
2. Créez un nouveau projet
3. Récupérez l'URL de connexion PostgreSQL

In [2]:
load_dotenv()
HOST= os.getenv("DB_HOST")
DATA_BASE= os.getenv("DATA_BASE")
USER= os.getenv("DB_USER")
PASSWORD=os.getenv("DB_PASSWORD")
PORT= os.getenv("DB_PORT")

In [3]:
# Configuration de connexion (à adapter selon votre provider)

DATABASE_CONFIG = {
    'host': HOST,  # Ou votre host Supabase
    'database': DATA_BASE,
    'user': USER,
    'password': PASSWORD,
    'port': PORT
}
# Création de l'engine SQLAlchemy
engine = create_engine(
    f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@"
    f"{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
)

# Test de connexion
def test_connection():
    """
    Testez votre connexion à la base

    Étapes :
    1. Utilisez pd.read_sql() pour exécuter "SELECT version()"
    2. Affichez la version PostgreSQL
    3. Gérez les erreurs de connexion
    """
    try:
        df_sql = pd.read_sql("SELECT version();", engine)
        print(df_sql)
    except Exception as e:
        print(f"Erreur de connexion : {e}")
        return False
    return True

In [4]:
test_connection()

                                             version
0  PostgreSQL 17.4 on aarch64-unknown-linux-gnu, ...


True


## Partie 2 : Import du dataset e-commerce

### 📊 Dataset Brazilian E-Commerce
Nous utilisons le célèbre dataset Olist (100k commandes réelles).

**Tables à créer :**
1. **customers** : customer_id, customer_city, customer_state
2. **orders** : order_id, customer_id, order_status, order_date, order_delivered_date
3. **order_items** : order_id, product_id, seller_id, price, freight_value
4. **products** : product_id, product_category, product_weight_g
5. **sellers** : seller_id, seller_city, seller_state

In [5]:
### 🗃️ Création des tables SQL
def create_tables():
    """
    Créez les tables dans PostgreSQL

    Tips :
    - Utilisez des SERIAL pour les IDs auto-increment
    - Ajoutez des index sur les clés étrangères
    - Incluez des contraintes de validation
    """

    create_customers = """    
    CREATE TABLE IF NOT EXISTS customers (
        customer_id VARCHAR(50) PRIMARY KEY,
        customer_city VARCHAR(100) NOT NULL,
        customer_state CHAR(2) NOT NULL
    );
    """
    create_orders = """ 
    CREATE TABLE IF NOT EXISTS orders (
        order_id VARCHAR(50) PRIMARY KEY,
        customer_id VARCHAR(50) NOT NULL,
        order_status VARCHAR(20) CHECK (order_status IN ('delivered', 'shipped', 'processing', 'canceled', 'invoiced', 'unavailable', 'created', 'approved')),
        order_date TIMESTAMP NOT NULL,
        order_delivered_date TIMESTAMP,
        FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
    );
    """
    create_order_itmes = """
    CREATE TABLE IF NOT EXISTS order_items (
        order_id VARCHAR(50) NOT NULL,
        order_item_id INTEGER NOT NULL,
        product_id VARCHAR(50) NOT NULL,
        seller_id VARCHAR(50) NOT NULL,
        price NUMERIC(10, 2),
        freight_value NUMERIC(10, 2),
        PRIMARY KEY (order_id, order_item_id),
        FOREIGN KEY (order_id) REFERENCES orders(order_id),
        FOREIGN KEY (product_id) REFERENCES products(product_id),
        FOREIGN KEY (seller_id) REFERENCES sellers(seller_id)
    );
    """
    create_product = """ 
    CREATE TABLE IF NOT EXISTS products (
        product_id VARCHAR(50) PRIMARY KEY,
        product_category VARCHAR(100),
        product_weight_g INT CHECK (product_weight_g >= 0)
    );
    """
    create_sellers = """ 
    CREATE TABLE IF NOT EXISTS sellers (
        seller_id VARCHAR(50) PRIMARY KEY,
        seller_city VARCHAR(100) NOT NULL,
        seller_state CHAR(2) NOT NULL
    );
    """
    # Complétez pour les autres tables
    # N'oubliez pas les contraintes de clés étrangères !

    with engine.connect() as conn:
        conn.execute(text(create_customers))     # 1
        conn.execute(text(create_product))       # 2
        conn.execute(text(create_sellers))       # 3
        conn.execute(text(create_orders))        # 4
        conn.execute(text(create_order_itmes))   # 5
        conn.commit()


In [6]:
create_tables()

######################################## Start data cleaning ################################################

In [7]:
# load all data frame to start data cleaning
df_customer= pd.read_csv("archive/olist_customers_dataset.csv")
df_product= pd.read_csv("archive/olist_products_dataset.csv")
df_sellers= pd.read_csv("archive/olist_sellers_dataset.csv")
df_order= pd.read_csv("archive/olist_orders_dataset.csv")
df_order_items= pd.read_csv("archive/olist_order_items_dataset.csv")

In [8]:
#  Drops specified columns from the DataFrame
def drop_columns(df, columns_to_drop):
    """
    Drops specified columns from the DataFrame.
    
    Parameters:
    - df: pandas DataFrame
    - columns_to_drop: list of column names to drop
    
    Returns:
    - DataFrame with specified columns removed
    """
    return df.drop(columns=columns_to_drop, errors='ignore')

In [9]:
# creat a list of columns to drop  
cols_to_drop = ['customer_unique_id','customer_zip_code_prefix', 'product_name_lenght','product_description_lenght', 'product_photos_qty',
            'product_length_cm', 'product_height_cm', 'product_width_cm', 'seller_zip_code_prefix', 'order_approved_at', 'order_delivered_carrier_date',
            'order_estimated_delivery_date', 'shipping_limit_date']
# run the function on each dataframe 
df_customer = drop_columns(df_customer, cols_to_drop)
df_product = drop_columns(df_product, cols_to_drop)
df_sellers = drop_columns(df_sellers, cols_to_drop)
df_order = drop_columns(df_order, cols_to_drop)
df_order_items = drop_columns(df_order_items, cols_to_drop)

In [10]:
# Rename specified columns 
def rename_columns(df, columns_mapping):
    """
    Renames columns in the DataFrame based on a dictionary mapping.
    
    Parameters:
    - df: pandas DataFrame
    - columns_mapping: dict of {old_name: new_name}
    
    Returns:
    - DataFrame with renamed columns
    """
    return df.rename(columns=columns_mapping)


In [11]:
# mapping list of columns to rename
cols_to_rename = {
    "product_category_name" : "product_category",
    "order_purchase_timestamp" : "order_date",
    "order_delivered_customer_date" : "order_delivered_date"
}
# Run the function on each data frame
df_customer = rename_columns(df_customer, cols_to_rename)
df_product = rename_columns(df_product, cols_to_rename)
df_sellers = rename_columns(df_sellers, cols_to_rename)
df_order = rename_columns(df_order, cols_to_rename)
df_order_items = rename_columns(df_order_items, cols_to_rename)

In [ ]:
# # Normalizes inconsistent values in the 'order_status' column
# def normalize_order_status(df):
#     """
#     Normalizes inconsistent values in the 'order_status' column.
#     """
#     df['order_status'] = df['order_status'].replace('canceled', 'cancelled')
#     return df
# df_order = normalize_order_status(df_order)

In [12]:
def clean_ecommerce_dataframes(dfs):
    """
    Clean and synchronize e-commerce DataFrames in pandas before database creation.
    
    Removes:
    - Rows with missing critical fields
    - Duplicates
    - Rows violating relationships (foreign key logic within pandas)
    
    Parameters:
    - dfs: dict containing the following DataFrames:
        'customers', 'orders', 'order_items', 'products', 'sellers'
    
    Returns:
    - cleaned_dfs: dict with cleaned DataFrames
    """
    
    # Unpack DataFrames
    customers = dfs.get('customers').copy()
    orders = dfs.get('orders').copy()
    order_items = dfs.get('order_items').copy()
    products = dfs.get('products').copy()
    sellers = dfs.get('sellers').copy()
    
    # 1. Clean customers
    customers.dropna(subset=['customer_id', 'customer_city', 'customer_state'], inplace=True)
    customers.drop_duplicates(subset=['customer_id'], inplace=True)
    
    # 2. Clean orders
    orders.dropna(subset=['order_id', 'customer_id', 'order_status'], inplace=True)
    orders.drop_duplicates(subset=['order_id'], inplace=True)
    orders = orders[orders['customer_id'].isin(customers['customer_id'])]
    
    # 3. Clean products
    products.dropna(subset=['product_id'], inplace=True)
    products.drop_duplicates(subset=['product_id'], inplace=True)
    
    # 4. Clean sellers
    sellers.dropna(subset=['seller_id'], inplace=True)
    sellers.drop_duplicates(subset=['seller_id'], inplace=True)
    
    # 5. Clean order_items
    order_items.dropna(subset=['order_id', 'order_item_id', 'product_id', 'seller_id', 'price', 'freight_value'], inplace=True)
    order_items.drop_duplicates(inplace=True)
    order_items = order_items[
        order_items['order_id'].isin(orders['order_id']) &
        order_items['product_id'].isin(products['product_id']) &
        order_items['seller_id'].isin(sellers['seller_id'])
    ]

    # Return cleaned DataFrames
    return {
        'customers': customers,
        'orders': orders,
        'order_items': order_items,
        'products': products,
        'sellers': sellers
    }


In [13]:
# load all data
dfs = {
    'customers': df_customer,
    'orders': df_order,
    'order_items': df_order_items,
    'products':df_product,
    'sellers': df_sellers
}

# clean all columns 
cleaned_dfs = clean_ecommerce_dataframes(dfs)

# print an example to check result
print(cleaned_dfs['order_items'].head())


                           order_id  order_item_id  \
0  00010242fe8c5a6d1ba2dd792cb16214              1   
1  00018f77f2f0320c557190d7a144bdd3              1   
2  000229ec398224ef6ca0657da4fc703e              1   
3  00024acbcdf0a6daa1e931b038114c75              1   
4  00042b26cf59d7ce69dfabb4e55b4fd9              1   

                         product_id                         seller_id   price  \
0  4244733e06e7ecb4970a6e2683c13e61  48436dade18ac8b2bce089ec2a041202   58.90   
1  e5f2d52b802189ee658865ca93d83a8f  dd7ddc04e1b6c2c614352b383efe2d36  239.90   
2  c777355d18b72b67abbeef9df44fd0fd  5b51032eddd242adc84c38acab88f23d  199.00   
3  7634da152a4610f1595efa32f14722fc  9d7a1d34a5052409006425275ba1c2b4   12.99   
4  ac6c3623068f30de03045865e4e10089  df560393f3a51e74553ab94004ba5c87  199.90   

   freight_value  
0          13.29  
1          19.93  
2          17.87  
3          12.79  
4          18.14  


#####################################--Inserts a DataFrame into a PostgreSQL table using SQLAlchemy--#################################


In [16]:
# function to insert the dataframe into data base 
def insert_df_to_db(df, table_name, engine):
    """
    Inserts a DataFrame into a PostgreSQL table using SQLAlchemy.
    
    Parameters:
    - df: pandas DataFrame to insert
    - table_name: str, the name of the target table in the DB
    - engine: SQLAlchemy engine object
    """
    try:
        df.to_sql(table_name, con=engine, if_exists='append', index=False)
        print(f"Inserted {len(df)} rows into '{table_name}' successfully.")
    except SQLAlchemyError as e:
        print(f"Failed to insert data into '{table_name}': {e}")
        

In [ ]:
# Run the function on each dataframe to insert data into the database

# insert_df_to_db(df_customer, "customers", engine)
# insert_df_to_db(df_product, "products", engine)
# insert_df_to_db(df_sellers, "sellers", engine)
# insert_df_to_db(df_order, "orders", engine)
# insert_df_to_db(df_order_items, "order_items", engine)

Inserted 99441 rows into 'customers' successfully.
Inserted 32951 rows into 'products' successfully.
Inserted 3095 rows into 'sellers' successfully.
Inserted 99441 rows into 'orders' successfully.
Inserted 112650 rows into 'order_items' successfully.


## Partie 3 : Requêtes SQL avancées


### 🔍 Analyses SQL à implémenter

#### 1. Analyse RFM (Récence, Fréquence, Montant)
```sql
-- Votre défi : Calculer les métriques RFM pour chaque client
WITH customer_metrics AS (
    SELECT
        c.customer_id,
        c.customer_state,
        -- Récence : jours depuis dernier achat
        -- Fréquence : nombre de commandes
        -- Montant : total dépensé
        
        -- Complétez cette requête CTE
        
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id, c.customer_state
)


```

In [26]:
rfm_query = """
--Calculate  RFM metrics (Recency, Frequency, Monetary)
WITH rfm_base AS (
    SELECT
        c.customer_id,
        DATE_PART('day', DATE '2018-10-17' - MAX(o.order_delivered_date)) AS recency,
        COUNT(o.order_id) AS frequency,
        SUM(oi.price) AS monetary
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
    JOIN order_items oi ON o.order_id = oi.order_id
    WHERE o.order_status = 'delivered'
    GROUP BY c.customer_id
),

--Assign RFM scores from 1 (worst) to 5 (best)
scored_rfm AS (
    SELECT
        customer_id,
        recency,
        frequency,
        monetary,
        NTILE(5) OVER (ORDER BY recency ASC) AS recency_score,   -- More recent = higher score
        NTILE(5) OVER (ORDER BY frequency DESC) AS frequency_score, -- More orders = higher score
        NTILE(5) OVER (ORDER BY monetary DESC) AS monetary_score    -- More spending = higher score
    FROM rfm_base
)

--Assign customer segments based on RFM scores
SELECT
    *,
    CASE
        WHEN recency_score >= 4 AND frequency_score >= 4 AND monetary_score >= 4 THEN 'Champions'
        WHEN recency_score >= 3 AND frequency_score >= 3 THEN 'Loyal Customers'
        WHEN recency_score >= 4 AND frequency_score BETWEEN 2 AND 3 THEN 'Potential Loyalists'
        WHEN recency_score BETWEEN 2 AND 3 AND frequency_score >= 3 THEN 'At Risk'
        WHEN recency_score <= 2 AND frequency_score <= 2 THEN 'Hibernating'
        WHEN recency_score = 1 THEN 'Lost'
        ELSE 'Others'
    END AS customer_segment
FROM scored_rfm;
"""
with engine.connect() as conn:
    df_rfm = pd.read_sql(text(rfm_query), conn)

df_rfm.head(15)


,customer_id,recency,frequency,monetary,recency_score,frequency_score,monetary_score,customer_segment
0,033fab69968b0d69099d64423831a236,0.0,1,26.40,1,2,5,Hibernating
1,27ae7c8a8fc20ce80d96f01b6f19961b,5.0,1,179.00,1,2,1,Hibernating
2,9e83d47684eb1a58b1c31830f5de10ac,14.0,1,70.00,1,4,3,Lost
3,ffa87b4246c4848711afb512bd51f161,19.0,1,209.99,1,1,1,Hibernating
4,1409b2945191b7aff1975ba2ce9918c5,21.0,1,49.90,1,3,4,Lost
5,7930549f156eea2b01b0fc2fdd323063,25.0,1,69.99,1,4,3,Lost
6,8be45a1114ff0e79615f7b8189aec7df,25.0,1,24.00,1,2,5,Hibernating
7,7c71fa0871e272a25eeccac52af90595,26.0,1,25.97,1,2,5,Hibernating
8,d306426abe5fca15e54b645e4462dc7b,27.0,1,144.99,1,5,2,Lost
9,a35878bee339b45240b5a327d933509b,27.0,1,29.99,1,3,5,Lost


In [ ]:
#### 2. Analyse géographique des ventes

def geographic_sales_analysis():
    """
    Analysez les performances par état/région

    Requêtes à écrire :
    1. Top 10 des états par CA
    2. Croissance MoM par région
    3. Taux de conversion par ville
    4. Distance moyenne vendeur-acheteur
    """

    query_top_states = """
    -- Votre requête SQL ici
    -- Utilisez des JOINs et GROUP BY
    -- Calculez le CA, nombre de commandes, panier moyen
    """

    return pd.read_sql(query_top_states, engine)

#### 3. Analyse temporelle et saisonnalité
```sql
-- Détectez les patterns saisonniers
SELECT
    EXTRACT(YEAR FROM order_date) as year,
    EXTRACT(MONTH FROM order_date) as month,
    EXTRACT(DOW FROM order_date) as day_of_week,
    COUNT(*) as order_count,
    SUM(price + freight_value) as total_revenue,
    AVG(price + freight_value) as avg_order_value
FROM orders o
JOIN order_items oi ON o.order_id = oi.order_id
WHERE order_status = 'delivered'
GROUP BY ROLLUP(
    EXTRACT(YEAR FROM order_date),
    EXTRACT(MONTH FROM order_date),
    EXTRACT(DOW FROM order_date)
)
ORDER BY year, month, day_of_week;
```

---

## Partie 4 : Analyse prédictive avec SQL

### 🔮 Modèles simples en SQL

In [ ]:
#### 1. Prédiction de churn

def churn_prediction_sql():
    """
    Identifiez les clients à risque de churn

    Indicateurs :
    - Pas d'achat depuis X jours
    - Baisse de fréquence d'achat
    - Diminution du panier moyen
    - Changement de comportement géographique
    """

    churn_query = """
    WITH customer_activity AS (
        -- Calculez les métriques d'activité récente
        -- Comparez avec l'historique du client
        -- Scorez le risque de churn
    )

    SELECT
        customer_id,
        days_since_last_order,
        order_frequency_trend,
        monetary_trend,
        churn_risk_score,
        CASE
            WHEN churn_risk_score > 0.7 THEN 'High Risk'
            WHEN churn_risk_score > 0.4 THEN 'Medium Risk'
            ELSE 'Low Risk'
        END as churn_segment
    FROM customer_activity;
    """

    return pd.read_sql(churn_query, engine)


#### 2. Recommandations produits
```sql
-- Market Basket Analysis simplifié
WITH product_pairs AS (
    SELECT
        oi1.product_id as product_a,
        oi2.product_id as product_b,
        COUNT(*) as co_purchase_count
    FROM order_items oi1
    JOIN order_items oi2 ON oi1.order_id = oi2.order_id
    WHERE oi1.product_id != oi2.product_id
    GROUP BY oi1.product_id, oi2.product_id
    HAVING COUNT(*) >= 10  -- Seuil minimum
)

SELECT
    product_a,
    product_b,
    co_purchase_count,
    co_purchase_count::float / total_a.count as confidence
FROM product_pairs pp
JOIN (
    SELECT product_id, COUNT(*) as count
    FROM order_items
    GROUP BY product_id
) total_a ON pp.product_a = total_a.product_id
ORDER BY confidence DESC;
```

---

## Partie 5 : Intégration avec les APIs météo

### 🌤️ Croisement données météo/ventes
```python
def weather_sales_correlation():
    """
    Correlez vos données météo du Notebook 1 avec les ventes
    
    Hypothèses à tester :
    1. Les ventes de certaines catégories augmentent-elles avec la pluie ?
    2. Y a-t-il un impact de la température sur les achats ?
    3. Les livraisons sont-elles impactées par la météo ?
    """
    
    # Récupérez les données météo historiques pour les villes brésiliennes
    weather_query = """
    SELECT DISTINCT customer_city, customer_state
    FROM customers
    WHERE customer_state IN ('SP', 'RJ', 'MG', 'RS', 'SC')
    ORDER BY customer_city;
    """
    
    cities = pd.read_sql(weather_query, engine)
    
    # Intégrez avec l'API météo
    # Analysez les corrélations
    
    pass
```

### 📊 Dashboard géo-temporel
```python
def create_geotemporal_dashboard():
    """
    Créez un dashboard interactif combinant :
    - Carte des ventes par région
    - Évolution temporelle avec météo
    - Segments clients géolocalisés
    - Prédictions par zone géographique
    """
    pass
```

---
## 🏆 Livrables finaux

### 📈 Rapport d'analyse complet
1. **Segmentation RFM (Recency, Frenquency, Monetary) ** : 5-7 segments avec caractéristiques
2. **Analyse géographique**  : Performances par région + recommandations
3. **Prédictions churn** : Liste des clients à risque + actions
4. **Recommandations produits** : Top 10 des associations
5. **Impact météo** : Corrélations significatives identifiées

### 🚀 Pipeline automatisé
```python
def automated_analysis_pipeline():
    """
    Pipeline qui :
    1. Se connecte à la DB
    2. Exécute toutes les analyses
    3. Met à jour les segments clients
    4. Génère le rapport automatiquement
    5. Envoie des alertes si nécessaire
    """
    pass
```

---

## 🎓 Auto-évaluation

- [ ] **Connexion DB** : PostgreSQL fonctionnelle
- [ ] **Requêtes complexes** : JOINs, CTEs, fonctions analytiques
- [ ] **Gestion des erreurs** : Connexions robustes
- [ ] **Performance** : Requêtes optimisées avec index
- [ ] **Intégration** : SQL + Python + APIs
- [ ] **Insights actionables** : Recommandations business claires

### 🔗 Préparation au Notebook 3
Le prochain notebook portera sur NoSQL (MongoDB) avec des données de réseaux sociaux et d'IoT, en temps réel.

### 💡 Bases de données alternatives
- **PlanetScale** : MySQL serverless gratuit
- **MongoDB Atlas** : 512MB gratuit
- **FaunaDB** : Base multi-modèle gratuite
- **Hasura Cloud** : GraphQL + PostgreSQL